# Data Load and Processing

In [112]:
import os
import glob
import sys
import configparser
import random
import cv2 as cv
import numpy as np
from keras.preprocessing.image import img_to_array, ImageDataGenerator

In [120]:
#drive folder path
drive_work_folder_path = ''
# cv.namedWindow("image", cv.WINDOW_NORMAL)
# cv.resizeWindow("image", 800, 500)

# np_oldopt = np.get_printoptions()
# np.set_printoptions(threshold=np.inf)

#
# file and folder names
#

#
# file and folder names
#
config = configparser.ConfigParser()
config.read(drive_work_folder_path+'codeconfig.ini')
infolder = drive_work_folder_path+config['path']['infolder']
outfolder = drive_work_folder_path+config['path']['outfolder']
img_out = config['path']['img_out']

extensions = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

imgoutfolder = os.path.join(outfolder + img_out.split("/")[0])

In [121]:
if (not os.path.exists(imgoutfolder)):
    os.makedirs(imgoutfolder)

inpath = glob.glob(infolder + "*.*")

n_classes = int(config['classinfo']['n_classes'])
clsnames = str(config['classinfo']['target_names'])

target_names = []
labels = []

for i in range(n_classes):
    target_names.append(clsnames.split(', ')[i])
    labels.append(float(i + 1))

In [122]:
filecount = 0

# flags
preproc_f = 0
rembg_f = 0
seg_f = 0
crop_f = 0
aug_f = 1
only_aug = 1


In [123]:
if(not only_aug):
    for f in inpath:
        name, ext = os.path.splitext(os.path.basename(f))

        if (str(ext).lower() not in extensions):
            continue

        file_number = (name.split('(', 1)[1]).split(')', 1)[0]
        filecount += 1

        print ("\n\t {}. processing {}").format(filecount, name)

        found = 0
        for i in range(n_classes):
            if (name.startswith(str(target_names[i]).lower())):
                class_number = labels[i]
                found = 1

        if (not found):
            print ("Wrong class - {}").format(class_number)
            sys.exit(0)

        #
        # read image
        #
        img = im1 = cv.imread(f)

        # image processing
        if(preproc_f):
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

            img_rgb_chanel = cv.split(img)
            img_hsv_chanel = cv.split(img_hsv)

        # remove background
        if(rembg_f):
            thresh_hsv_s = 0.28 * 255

            ret_s, hsv_s_thresh = cv.threshold(
                img_hsv_chanel[1], thresh_hsv_s, 255, cv.THRESH_BINARY)
            hsv_s_thresh = cv.GaussianBlur(hsv_s_thresh, (5, 5), 0)
            rembg = cv.bitwise_and(img, img, mask=hsv_s_thresh)

            # cv.imwrite(outfolder + img_out + name + "-rembg" + ".png", rembg)

        # TODO write comments

        # disease segmentation
        if (seg_f):
            hue = img_hsv_chanel[0]
            thresh_hsv_h = 22

            _, k = cv.threshold(hue, thresh_hsv_h, 255, cv.THRESH_BINARY)

            k = cv.medianBlur(k, 15)
            k_inv = cv.bitwise_not(k)

            seg = cv.bitwise_and(rembg, rembg, mask=k_inv)
            seg = cv.medianBlur(seg, 7)
            # cv.imwrite(outfolder + img_out + name + "-seg" + ".png", seg)

            seg_gray = cv.cvtColor(seg, cv.COLOR_RGB2GRAY)
            _, seg_bin = cv.threshold(seg_gray, 40, 255, cv.THRESH_BINARY)

            seg_bin = cv.copyMakeBorder(seg_bin, 10, 10, 10, 10,
                                        cv.BORDER_CONSTANT, (0, 0, 0))

            # cv.imwrite(outfolder + img_out + name + "-segbin" + ".png", seg_bin)

            new_segmask = np.zeros(seg_bin.shape, np.uint8)

            seg_canny = cv.Canny(seg_bin, 100, 200)

            contours, hierarchy = cv.findContours(
                seg_canny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            if(len(contours)):
                sorted_cnts = sorted(
                    contours, key=cv.contourArea, reverse=True)
                cnt_max = cv.contourArea(sorted_cnts[0])

                for c, e in enumerate(contours):
                    area = cv.contourArea(contours[c])
                    if (area >= cnt_max*0.1):
                        cv.drawContours(new_segmask, [contours[c]], 0, 255, -1)

            rb = rembg.copy()
            rb = cv.copyMakeBorder(rb, 10, 10, 10, 10,
                                   cv.BORDER_CONSTANT, (0, 0, 0))
            seg = cv.bitwise_and(rb, rb, mask=new_segmask)
            # cv.imwrite(outfolder + img_out + name + "-newseg" + ".png", seg)

        # cropping images
        if (crop_f):
            # points are col x row
            # images are row x col

            seg_bin = new_segmask
            segline = seg.copy()

            row_y = seg_bin.shape[0]
            col_x = seg_bin.shape[1]

            left = seg_bin.shape[1]
            p_left1 = (0, 0)
            p_left2 = (0, 0)

            for y in range(0, row_y):
                for x in range(0, col_x):
                    if (seg_bin[y][x]):
                        if (x <= left):
                            left = x

            p_left1 = (left-2, 0)
            p_left2 = (left-2, col_x-2)
            # left_line = cv.line(segline, p_left1, p_left2, (255, 0, 0), 1)

            # print("Left = {} to {}").format(p_left1, p_left2)
            # cv.imwrite(outfolder + img_out + name + "-cropbox" + ".png", left_line)

            top = seg_bin.shape[0]
            p_top1 = (0, 0)
            p_top2 = (0, 0)

            for x in range(col_x-1, 0, -1):
                for y in range(row_y-1, 0, -1):
                    if (seg_bin[y][x]):
                        if (y <= top):
                            top = y

            p_top1 = (0, top-2)
            p_top2 = (col_x-2, top-2)
            # top_line = cv.line(segline, p_top1, p_top2, (255, 0, 0), 1)

            # print("Top = {} to {}").format(p_top1, p_top2)
            # cv.imwrite(outfolder + img_out + name + "-cropbox" + ".png", top_line)

            right = 0
            p_right1 = (0, 0)
            p_right2 = (0, 0)

            for y in range(row_y-1, 0, -1):
                for x in range(col_x-1, 0, -1):
                    if (seg_bin[y][x]):
                        if (x >= right):
                            right = x

            p_right1 = (right+2, 0)
            p_right2 = (right+2, col_x+2)
            # right_line = cv.line(segline, p_right1, p_right2, (255, 0, 0), 1)

            # print("Right = {} to {}").format(p_right1, p_right2)
            # cv.imwrite(outfolder + img_out + name + "-cropbox" + ".png", right_line)

            bottom = 0
            p_bottom1 = (0, 0)
            p_bottom2 = (0, 0)

            for x in range(0, col_x):
                for y in range(0, row_y):
                    if (seg_bin[y][x]):
                        if (y >= bottom):
                            bottom = y

            p_bottom1 = (0, bottom+2)
            p_bottom2 = (col_x+2, bottom+2)
            # bottom_line = cv.line(segline, p_bottom1, p_bottom2, (255, 0, 0), 1)

            # print("Bottom = {} to {}").format(p_bottom1, p_bottom2)
            # cv.imwrite(outfolder + img_out + name + "-croplines" + ".png", bottom_line)

            # intersection of lines
            x_tl = (p_left1[0], p_top1[1])
            x_tr = (p_right1[0], p_top2[1])
            x_bl = (p_left2[0], p_bottom1[1])
            x_br = (p_right2[0], p_bottom2[1])

            # test the intersections
            # bottom_line[x_bl[1]][x_bl[0]] = (255, 255, 255)
            # cv.imwrite(outfolder + img_out + name + "-cross" + ".png", bottom_line)

            # print("Top-left point = {} \nBottom-right point = {}").format(x_tl, x_br)
            # cropbox = cv.rectangle(seg, x_tl, x_br, (0,0,0), 1)
            # cv.imwrite(outfolder + img_out + name + "-cropbox" + ".png", cropbox)

            cropped = seg.copy()
            cropped = cropped[x_tl[1]:x_br[1], x_tl[0]:x_br[0]]
            cropped = cv.copyMakeBorder(cropped, 10, 10, 10, 10,
                                        cv.BORDER_CONSTANT, (0, 0, 0))
            # cv.imwrite(outfolder + img_out + name + "-cropped" + ".png", cropped)
            cv.imwrite(outfolder + img_out + name + ".png", cropped)

In [125]:

#
# MAIN CODE BEGINS
#

# image augmentation
if(aug_f):
    inpath = glob.glob(infolder + "/" + "*.*")
    data_aug = os.path.join(outfolder + "aug/")
    if (not os.path.exists(data_aug)):
        os.makedirs(data_aug)

    def motion_blur(img, angle, deg):
        rotation = cv.getRotationMatrix2D((deg / 2, deg / 2), angle, 1)

        kernel = np.diag(np.ones(deg))
        kernel = (cv.warpAffine(kernel, rotation, (deg, deg))) / deg

        res = cv.filter2D(img, -1, kernel)
        cv.normalize(res, res, 0, 255, cv.NORM_MINMAX)
        res = np.array(res, dtype=np.uint8)
        return res

    for f in inpath:
        name, ext = os.path.splitext(os.path.basename(f))

        print("\n\tAugmenting {}".format(name))

        img_orig = cv.imread(f)
        img = cv.resize(img_orig, (512, 512), interpolation=cv.INTER_AREA)
        data = img_to_array(img)
        samples = np.expand_dims(data, axis=0)

        aug = ImageDataGenerator(
            horizontal_flip=True,
            rotation_range=90, shear_range=0.15,
            width_shift_range=0.2, height_shift_range=0.2,
            zoom_range=[0.5, 2], brightness_range=[0.25, 1.5],
            fill_mode="nearest")

        gen = aug.flow(samples)

        for i in range(25):
            im = gen.next()
            im = im.reshape((512, 512, 3))

            # motion blur randomly, about 30%  data
            rand = random.randint(0, 2)
            if (not rand):
                im = motion_blur(im, 50, 16)

            cv.imwrite(data_aug + name + "#" + str(i) + ".png", im)


print("COMPLETED")



	Augmenting blb (15)

	Augmenting blb (12)

	Augmenting blb (1)

	Augmenting blb (10)

	Augmenting blb (11)

	Augmenting blb (13)

	Augmenting blb (14)

	Augmenting blb (16)

	Augmenting blb (23)

	Augmenting blb (20)

	Augmenting blb (2)

	Augmenting blb (17)

	Augmenting blb (22)

	Augmenting blb (21)

	Augmenting blb (24)

	Augmenting blb (19)

	Augmenting blb (18)

	Augmenting blb (3)

	Augmenting blb (31)

	Augmenting blb (28)

	Augmenting blb (27)

	Augmenting blb (30)

	Augmenting blb (26)

	Augmenting blb (25)

	Augmenting blb (29)

	Augmenting blb (35)

	Augmenting blb (33)

	Augmenting blb (37)

	Augmenting blb (36)

	Augmenting blb (38)

	Augmenting blb (34)

	Augmenting blb (32)

	Augmenting blb (6)

	Augmenting blb (7)

	Augmenting blb (4)

	Augmenting blb (40)

	Augmenting blb (5)

	Augmenting blb (39)

	Augmenting blb (9)

	Augmenting bs (1)

	Augmenting blb (8)

	Augmenting bs (10)

	Augmenting bs (11)

	Augmenting bs (12)

	Augmenting bs (13)

	Augmenting bs (15)

	Au

# Feature Extraction

In [126]:
import os
import glob
import sys
import shutil
import pickle as cPickle
import configparser
import time
import datetime as dt
import cv2 as cv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import kurtosis, skew
from skimage.feature import greycomatrix, greycoprops, local_binary_pattern

In [127]:
# timing and timestamping functions
genesis = time.time()  # timing
timestamp = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# cv.namedWindow("image", cv.WINDOW_NORMAL)
# cv.resizeWindow("image", 800, 500)

#np_oldopt = np.get_printoptions()
# np.set_printoptions(threshold=np.inf)

#drive folder path
drive_work_folder_path = ''
# cv.namedWindow("image", cv.WINDOW_NORMAL)
# cv.resizeWindow("image", 800, 500)

# np_oldopt = np.get_printoptions()
# np.set_printoptions(threshold=np.inf)

#
# file and folder names
#

#
# file and folder names
#
config = configparser.ConfigParser()
config.read(drive_work_folder_path+'codeconfig.ini')
infolder = drive_work_folder_path+config['path']['infolder']
outfolder = drive_work_folder_path+config['path']['outfolder']
img_out = config['path']['img_out']

extensions = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

imgoutfolder = os.path.join(outfolder + img_out.split("/")[0])

In [128]:
if (not os.path.exists(imgoutfolder)):
    os.makedirs(imgoutfolder)

inpath = glob.glob(infolder + "*.*")

fextout = os.path.join(outfolder + 'FEXT-' + timestamp)
os.makedirs(fextout)

feat_list = open(fextout + '/' + 'Feature_List-' + timestamp + '.txt', 'w')
colout = open(fextout + '/' + 'FL-' + timestamp + '-feature columns.txt', 'w')

In [129]:

FL = []
flname = []
flname_on = 1
csv_idx = []
csv_idx_on = 1
filecount = 0

n_classes = int(config['classinfo']['n_classes'])
clsnames = str(config['classinfo']['target_names'])

target_names = []
labels = []

for i in range(n_classes):
    target_names.append(clsnames.split(', ')[i])
    labels.append(float(i + 1))

#
# flags
#
resize_f = 0

color_f = 1
shape_f = 1
texture_f = 1

mb_f = 1
mg_f = 1
mr_f = 1
mh_f = 1
ms_f = 1
mv_f = 0
sdb_f = 1
sdg_f = 1
sdr_f = 1
bghist_f = (16 * 16)
hshist_f = (16 * 16)  # bin * bin
skw_f = 1
kurt_f = 1
sdseg_f = 1

cntperi_f = 1
cntarea_f = 1
hum_f = 7  # 7 moments

glcm_f = 4 * 1  # mean of 4 glcm, 4 angle
lbphist_f = 32  # 32  # bins

# flags finished


In [130]:

# calculate feature number for feature list shaping
prid_f_num = 1
class_f_num = 1

if (color_f):
    color_f_num = mb_f + mg_f + mr_f + mh_f + ms_f + mv_f + \
        sdb_f + sdg_f + sdr_f + bghist_f + hshist_f + skw_f + kurt_f + sdseg_f
else:
    color_f_num = 0

if (shape_f):
    shape_f_num = cntperi_f + cntarea_f + hum_f
else:
    shape_f_num = 0

if (texture_f):
    texture_f_num = glcm_f + lbphist_f
else:
    texture_f_num = 0

feat_num = prid_f_num + class_f_num + color_f_num + \
    shape_f_num + texture_f_num

#
# MAIN CODE BEGINS
#

print("\n Beginning feature extraction... \n")


fext_gen = time.time()

for f in inpath:
    name, ext = os.path.splitext(os.path.basename(f))

    if (str(ext).lower() not in extensions):
        continue

    file_number = (name.split('(', 1)[1]).split(')', 1)[0]
    filecount += 1

    print("\n\t {}. processing {}".format(filecount, name))

    found = 0
    for i in range(n_classes):
        if (name.startswith(str(target_names[i]).lower())):
            class_number = labels[i]
            found = 1

    if (not found):
        print("Wrong class - {}".format(class_number))
        sys.exit(0)

    # processing_id = classnum(1) + file_num
    processing_id = int(str(int(class_number)) + str(file_number))
    FL.append(processing_id)

    if flname_on:
        flname.append("Processing ID")
    if csv_idx_on:
        csv_idx.append("Processing ID")

    FL.append(class_number)

    if flname_on:
        flname.append("Class Label")
    if csv_idx_on:
        csv_idx.append("Class Label")

    #
    # read images
    #
    img = im1 = cv.imread(f)

    if (resize_f):
        res_row = min(512, img.shape[1])
        res_col = min(256, img.shape[0])
        # res_row = res_col = 512
        img = cv.resize(im1, (res_row, res_col),
                        interpolation=cv.INTER_AREA)

    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    img_rgb_chanel = cv.split(img)
    img_hsv_chanel = cv.split(img_hsv)

    # feature extraction
    seg = img
    segf = seg.flatten()
    seg_gray = cv.cvtColor(seg, cv.COLOR_RGB2GRAY)

    if (color_f):
        print("Extracting color features...")

        b_seg, g_seg, r_seg = cv.split(seg)

        b_segf = b_seg.flatten()
        g_segf = g_seg.flatten()
        r_segf = r_seg.flatten()
        mean_r_seg = mean_g_seg = mean_b_seg = 0

        if (mb_f):
            if flname_on:
                flname.append("Mean of RGB-Blue of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of RGB-Blue")

            if np.count_nonzero(b_segf) != 0:
                mean_b_seg = sum(b_segf[np.nonzero(b_segf)]) / \
                    np.count_nonzero(b_segf)  # FEX mean_b_seg
                FL.append(mean_b_seg)
            else:
                FL.append(0.0)

        if (mg_f):
            if flname_on:
                flname.append("Mean of RGB-Green of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of RGB-Green")

            if np.count_nonzero(g_segf) != 0:
                mean_g_seg = sum(g_segf[np.nonzero(g_segf)]) / \
                    np.count_nonzero(g_segf)  # FEX mean_g_seg
                FL.append(mean_g_seg)
            else:
                FL.append(0.0)

        if (mr_f):
            if flname_on:
                flname.append("Mean of RGB-Red of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of RGB-Red")

            if np.count_nonzero(r_segf) != 0:
                mean_r_seg = sum(r_segf[np.nonzero(r_segf)]) / \
                    np.count_nonzero(r_segf)  # FEX mean_r_seg
                FL.append(mean_r_seg)
            else:
                FL.append(0.0)

        seg_hsv = cv.cvtColor(seg, cv.COLOR_RGB2HSV)
        h_seg, s_seg, v_seg = cv.split(seg_hsv)

        h_segf = h_seg.flatten()
        s_segf = s_seg.flatten()
        v_segf = v_seg.flatten()
        mean_h_seg = mean_s_seg = mean_v_seg = 0

        if (mh_f):
            if flname_on:
                flname.append("Mean of HSV-Hue of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of HSV-Hue")

            if np.count_nonzero(h_segf) != 0:
                mean_h_seg = sum(h_segf[np.nonzero(h_segf)]) / \
                    np.count_nonzero(h_segf)  # FEX mean_h_seg
                FL.append(mean_h_seg)
            else:
                FL.append(0.0)

        if (ms_f):
            if flname_on:
                flname.append("Mean of HSV-Saturation of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of HSV-Saturation")

            if np.count_nonzero(s_segf) != 0:
                mean_s_seg = sum(s_segf[np.nonzero(s_segf)]) / \
                    np.count_nonzero(s_segf)  # FEX mean_s_seg
                FL.append(mean_s_seg)
            else:
                FL.append(0.0)

        if (mv_f):
            if flname_on:
                flname.append("Mean of HSV-Value of segmented image")
            if csv_idx_on:
                csv_idx.append("Mean of HSV-Value")

            if np.count_nonzero(v_segf) != 0:
                mean_v_seg = sum(v_segf[np.nonzero(v_segf)]) / \
                    np.count_nonzero(v_segf)  # FEX mean_v_seg
                FL.append(mean_v_seg)
            else:
                FL.append(0.0)

        # standard deviation
        if (sdb_f):
            b_seg_std = np.std(b_segf)  # FEX b_seg_std
            FL.append(b_seg_std)
            if flname_on:
                flname.append(
                    "Standard Deviation of RGB-Blue of segmented image")
            if csv_idx_on:
                csv_idx.append("SD of RGB-Blue")

        if (sdg_f):
            g_seg_std = np.std(g_segf)  # FEX g_seg_std
            FL.append(g_seg_std)
            if flname_on:
                flname.append(
                    "Standard Deviation of RGB-Green of segmented image")
            if csv_idx_on:
                csv_idx.append("SD of RGB-Green")

        if (sdr_f):
            r_seg_std = np.std(r_segf)  # FEX r_seg_std
            FL.append(r_seg_std)
            if flname_on:
                flname.append(
                    "Standard Deviation of RGB-Red of segmented image")
            if csv_idx_on:
                csv_idx.append("SD of RGB-Red")

        if (bghist_f):
            bghistbins = 16
            total_bins = bghistbins * bghistbins
            img_rgb_hist_bg, img_rgb_hist_bg_edge1, img_rgb_hist_bg_edge2 = np.histogram2d(
                b_segf, g_segf, bins=bghistbins, range=((0, 255), (0, 255)))

            img_rgb_hist_bgf = np.ravel(img_rgb_hist_bg)

            # min-max normalization = (x - min)/(max - min)
            img_rgb_hist_bg_norm = (img_rgb_hist_bgf - min(img_rgb_hist_bgf)) / (
                max(img_rgb_hist_bgf) - min(img_rgb_hist_bgf))  # FEX img_rgb_hist_bg_norm

            FL.extend(np.ravel(img_rgb_hist_bg_norm))
            if flname_on:
                flname.append(
                    "min-max normalized Blue-Green Histogram (16*16 bins)")

            # writing feature column names
            if csv_idx_on:
                for i in range(total_bins):
                    csv_idx.append("BG-Hist-" + str(i))

        if (hshist_f):
            hshistbins = 16
            total_bins = hshistbins * hshistbins
            img_hsv_hist_hs, img_hsv_hist_hs_edge1, img_hsv_hist_hs_edge2 = np.histogram2d(
                img_hsv_chanel[0].flatten(), img_hsv_chanel[1].flatten(), bins=hshistbins, range=((0, 180), (0, 256)))

            img_hsv_hist_hsf = np.ravel(img_hsv_hist_hs)

            # min-max normalization = (x - min)/(max - min)
            img_hsv_hist_hs_norm = (img_hsv_hist_hsf - min(img_hsv_hist_hsf)) / (
                max(img_hsv_hist_hsf) - min(img_hsv_hist_hsf))  # FEX img_hsv_hist_hs_norm

            FL.extend(np.ravel(img_hsv_hist_hs_norm))
            if flname_on:
                flname.append(
                    "min-max normalized Hue-Saturation Histogram (16*16 bins)")

            # writing feature column names
            if csv_idx_on:
                for i in range(total_bins):
                    csv_idx.append("HS-Hist-" + str(i))

        # processing for contour detection
        ret1, seg_bin = cv.threshold(seg_gray, 40, 255, cv.THRESH_BINARY)
        seg_bin = cv.medianBlur(seg_bin, 5)
        seg_canny = cv.Canny(seg_bin, 100, 200)
        # cv.imwrite(outfolder+ img_out + name + "-seg_canny" + ".png", seg_canny)

        contours, hierarchy = cv.findContours(
            seg_canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        # seg_cnt = cv.drawContours(seg, contours, -1, (0, 0, 255), 2)
        # cv.imwrite(outfolder + img_out + name + "-seg_cntdraw" + ".png", seg_cnt)

        #
        # find contour pixel coordinates and intensity
        #
        cnt_intensity = []
        cnt_coord = []

        for c, e in enumerate(contours):
            mask = np.zeros(seg_canny.shape, np.uint8)
            cv.drawContours(mask, [contours[c]], 0, 255, -1)
            pixelpoints = np.transpose(np.nonzero(mask))  # contour coordinates
            cnt_coord.extend(pixelpoints)

            for p, e in enumerate(pixelpoints):
                # intensity at the contour coordinates
                intensity = seg_gray[pixelpoints[p][0]][pixelpoints[p][1]]
                if (intensity):
                    cnt_intensity.append(intensity)

        if (skw_f):
            seg_skew = skew(cnt_intensity, None)  # FEX seg_skew
            FL.append(seg_skew)
            if flname_on:
                flname.append("Skewness of diseased segment")
            if csv_idx_on:
                csv_idx.append("Skewness")

        if (kurt_f):
            # kurtosis (pearson)     #FEX seg_kurt
            seg_kurt = kurtosis(cnt_intensity, None, fisher=False)
            FL.append(seg_kurt)
            if flname_on:
                flname.append("Kurtosis of diseased segment")
            if csv_idx_on:
                csv_idx.append("Kurtosis")

        if (sdseg_f):
            seg_cnt_std = np.std(cnt_intensity)
            FL.append(seg_cnt_std)
            if flname_on:
                flname.append("Standard Deviation of diseased segment")
            if csv_idx_on:
                csv_idx.append("SD of diseased segment")

    if (shape_f):
        print("Extracting shape features...")

        if (cntperi_f):
            cnt_peri = []

            for c, e in enumerate(contours):
                perimeter = cv.arcLength(contours[c], True)
                cnt_peri.append(perimeter)

            # removing contours having less than 30% perimeter of the biggest contour
            if (np.count_nonzero(cnt_peri) != 0):
                max_cntp = max(cnt_peri)
            else:
                max_cntp = 0.0

            cnt_perimeter = []

            for i, e in enumerate(cnt_peri):
                if (cnt_peri[i] > (max_cntp * 0.3)):
                    cnt_perimeter.append(cnt_peri[i])

            if cnt_perimeter:
                cnt_perimeter_avg = sum(cnt_perimeter) / len(cnt_perimeter)
            else:
                cnt_perimeter_avg = 0.0

            FL.append(cnt_perimeter_avg)  # FEX cnt_perimeter_avg
            if flname_on:
                flname.append("Average contour perimeter")
            if csv_idx_on:
                csv_idx.append("Average contour perimeter")

        if (cntarea_f):
            cnt_ar = []

            for c, e in enumerate(contours):
                area = cv.contourArea(contours[c])
                cnt_ar.append(area)

            # removing contours having less than 30% area of the biggest contour
            if (np.count_nonzero(cnt_ar) != 0):
                max_cnt = max(cnt_ar)
            else:
                max_cnt = 0.0

            cnt_area = []

            for i, e in enumerate(cnt_ar):
                if (cnt_ar[i] > (max_cnt * 0.3)):
                    cnt_area.append(cnt_ar[i])

            if cnt_area:
                cnt_area_avg = sum(cnt_area) / len(cnt_area)
            else:
                cnt_area_avg = 0.0

            FL.append(cnt_area_avg)  # FEX cnt_area_avg
            if flname_on:
                flname.append("Average contour area")
            if csv_idx_on:
                csv_idx.append("Average contour area")

        if (hum_f):
            mmnt = cv.moments(seg_bin)
            hu_m = cv.HuMoments(mmnt)

            FL.extend(np.ravel(hu_m))  # FEX hu_mmnt
            if flname_on:
                flname.append("Hu Moments (7)")
            if csv_idx_on:
                for i in range(7):
                    csv_idx.append("Hu moment-" + str(i))

    if (texture_f):
        print("Extracting texture features...")

        if (glcm_f):
            # mean of 4 GLCM features in 0, 45 (np.pi / 4), 90 (np.pi / 2), 135 (3 * np.pi / 4) degrees
            glcm_seg = greycomatrix(
                seg_gray, [3], [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], normed=True)

            contrast = greycoprops(glcm_seg, 'contrast')  # FEX contrast

            contrast = contrast.mean()
            FL.append(contrast)

            # use the commented version for individual values (not mean)
            # contrast = [contrast[0][0], contrast[0][1], contrast[0][2], contrast[0][3]]
            # FL.extend(contrast)
            # if flname_on: flname.append("GLCM [at distance=3, angles=0,45,90,135]  Contrast")

            if flname_on:
                flname.append(
                    "GLCM [at distance=3, mean, angles=0,45,90,135]  Contrast")
            if csv_idx_on:
                csv_idx.append("GLCM Contrast")

            correlation = greycoprops(
                glcm_seg, 'correlation')  # FEX correlation

            correlation = correlation.mean()
            FL.append(correlation)

            # correlation = [correlation[0][0], correlation[0][1], correlation[0][2], correlation[0][3]]
            # FL.extend(correlation)

            if flname_on:
                flname.append("GLCM Correlation")
            if csv_idx_on:
                csv_idx.append("GLCM Correlation")

            energy = greycoprops(glcm_seg, 'energy')  # FEX energy

            energy = energy.mean()
            FL.append(energy)

            # energy = [energy[0][0], energy[0][1], energy[0][2], energy[0][3]]
            # FL.extend(energy)

            if flname_on:
                flname.append("GLCM Energy")
            if csv_idx_on:
                csv_idx.append("GLCM Energy")

            homogeneity = greycoprops(
                glcm_seg, 'homogeneity')  # FEX homogeneity

            homogeneity = homogeneity.mean()
            FL.append(homogeneity)

            # homogeneity = [homogeneity[0][0], homogeneity[0][1], homogeneity[0][2], homogeneity[0][3]]
            # FL.extend(homogeneity)

            if flname_on:
                flname.append("GLCM Homogeneity")
            if csv_idx_on:
                csv_idx.append("GLCM Homogeneity")

        if (lbphist_f):
            # normalized LBP (P, R = 24, 3.0) histogram
            # print "Calculating normalized LBP (P,R = 24, 3.0) histogram..."

            R = 3.0
            P = int(8 * R)
            lbphbins = 32

            lbp_seg = local_binary_pattern(seg_gray, P, R, method='uniform')

            lbp_seg_hist, lbp_seg_hist_edges = np.histogram(
                lbp_seg, bins=lbphbins, range=(0, 256))

            lbp_seg_hist = lbp_seg_hist.astype("float")
            lbp_seg_hist_norm = (lbp_seg_hist - min(lbp_seg_hist)) / \
                (max(lbp_seg_hist) - min(lbp_seg_hist))

            FL.extend(np.ravel(lbp_seg_hist_norm))
            if flname_on:
                flname.append(
                    "min-max normalized LBP (P, R = 24, 3.0) histogram (32 bins)")
            if csv_idx_on:
                for i in range(lbphbins):
                    csv_idx.append("LBP-Hist-" + str(i))

            # cv.imwrite(outfolder + img_out + name + "-lbp_seg" + ".png", lbp_seg.astype("uint8"))

    # formatting feature list
    # NaN values are replaced with 0
    FL = [0.0 if np.isnan(x) else x for x in FL]
#    FL = [round(x, 6) for x in FL]

    if (not feat_list.closed):
        feat_list.write(str(flname))
        total_feature = len(flname) - 2

        if (hum_f):
            total_feature += 6

        print("\n \n Total number of features: {}".format(
            total_feature),feat_list)

        print("\n \n Dataset ID: {}".format(
            str(config['info']['dataset']).encode('ascii', 'ignore')),feat_list)

        if (resize_f):
            print("\n Images downsized to: 512x256 or lower",feat_list)

        feat_list.close()
        flname_on = 0

    if (not colout.closed):
        colout.write(str(csv_idx))
        colout.close()
        csv_idx_on = 0

    # dump features modularly
    if (filecount % 50 == 0):
        # save in temporary file for modular processing of large number of files
        with open(fextout + '/' + 'fltmp-' + str(filecount) + '.pkl', 'ab+') as tmpfile:
            cPickle.dump(FL, tmpfile, protocol=cPickle.HIGHEST_PROTOCOL)

        FL = []

    # print ("Length of FL (feature vector) = {}").format(len(FL))



 Beginning feature extraction... 


	 1. processing blb (15)
Extracting color features...
Extracting shape features...
Extracting texture features...


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag


 
 Total number of features: 27 <_io.TextIOWrapper name='/content/drive/MyDrive/og_cropped/output/FEXT-2023-10-26_10-01-01/Feature_List-2023-10-26_10-01-01.txt' mode='w' encoding='UTF-8'>

 
 Dataset ID: b'rice-leaf-by-Prajapati-et-al' <_io.TextIOWrapper name='/content/drive/MyDrive/og_cropped/output/FEXT-2023-10-26_10-01-01/Feature_List-2023-10-26_10-01-01.txt' mode='w' encoding='UTF-8'>

	 2. processing blb (12)
Extracting color features...
Extracting shape features...
Extracting texture features...


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag


	 3. processing blb (1)
Extracting color features...
Extracting shape features...
Extracting texture features...


...

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag


	 119. processing ls (8)
Extracting color features...
Extracting shape features...
Extracting texture features...


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag


	 120. processing ls (9)
Extracting color features...
Extracting shape features...
Extracting texture features...


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

In [131]:
# dump the rest of the features
tmpfile = open(fextout + '/' + 'fltmp-' + str(filecount + 1) + '.pkl', 'ab+')
cPickle.dump(FL, tmpfile, protocol=cPickle.HIGHEST_PROTOCOL)
tmpfile.close()

# print ("\n Number of feature column = {}").format(feat_num)

# concateate all the temprorary feature files
tmppath = glob.glob(fextout + "/" + "*.*")
FL = []

In [132]:
for f in tmppath:
    name, ext = os.path.splitext(os.path.basename(f))
    if (str(name).startswith("fltmp-")):
        with open(f, "rb") as ffile:
            FL.extend(cPickle.load(ffile))
        os.remove(f)

In [133]:

feat = np.reshape(FL, (-1, int(feat_num)))

df = pd.DataFrame(feat, columns=csv_idx)
with open(fextout + '/' + 'FL-' + timestamp + '.csv', 'w') as csvout:
    df.to_csv(csvout, sep=',', mode='w', decimal='.')

with open(fextout + '/' + 'FL-' + timestamp + '.pkl', 'wb') as outfile:
    cPickle.dump(feat, outfile, protocol=cPickle.HIGHEST_PROTOCOL)

outfile.close()

# remove previous files

remfile = glob.glob(outfolder + '*.*')
for f in remfile:
    os.remove(f)

with open(outfolder + 'feature columns.txt', 'w') as colout:
    colout.write(str(csv_idx))


shutil.copy2(fextout + '/' + 'FL-' + timestamp + '.pkl',
             outfolder + 'FL-' + timestamp + '.pkl')
shutil.copy2(fextout + '/' + 'Feature_List-' + timestamp + '.txt',
             outfolder + 'Feature_List-' + timestamp + '.txt')

# cleanup

fext_time = time.time() - fext_gen  # timing finished
print('\n Feature extraction completed in::: %.2fs.' % fext_time)

# cv.waitKey(0)
# cv.destroyAllWindows()

armageddon = time.time() - genesis  # timing finished

print('\n Total time: %.3fs.' % armageddon)
print("\n END OF CODE")

# run classifier
#execfile('ms-thesis -CLF.py')



 Feature extraction completed in::: 408.20s.

 Total time: 417.506s.

 END OF CODE


# CLF

In [135]:
!pip install latex

In [72]:
import os
import glob
import sys
import pickle as cPickle
import configparser
import random
import time
import datetime as dt
import numpy as np
import xgboost as xgb
from matplotlib import cm
from matplotlib import pyplot as plt
from prettytable import PrettyTable as prt
from sklearn import metrics, svm
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [136]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [73]:

#drive folder path
drive_work_folder_path = ''

# timing and timestamping functions
genesis = time.time()
timestamp = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

#
# file and folder names
#
config = configparser.ConfigParser()
config.read(drive_work_folder_path+'codeconfig.ini')
infolder = drive_work_folder_path+config['path']['infolder']
outfolder = drive_work_folder_path+config['path']['outfolder']

In [137]:
feat_fname = glob.glob(outfolder + 'FL-*.pkl')

In [74]:
# process the filename string of feature file
#
feat_fname = str(feat_fname[0])
feat_fname = feat_fname.split('/')[-1]
feat_file = feat_fname.split('.', 1)[0]

In [75]:
with open(outfolder + feat_fname, 'rb') as infile:
  u = cPickle._Unpickler(infile)
  u.encoding = 'latin1'
  feat_in = u.load()

with open(outfolder + 'feature columns.txt', "r") as inp:
  fcols = inp.read()

In [76]:
# matplotlib plotting global parameters
plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#
# flags
#
svm_f = 0
xgb_f = 1

# global variable for comparison
SVM_ACC = 0.0
SVM_CV = 0.0
XGB_ACC = 0.0
XGB_CV = 0.0

#
# class info
#
n_classes = int(config['classinfo']['n_classes'])
clsnames = str(config['classinfo']['target_names'])

target_names = []
labels = []


In [77]:

for i in range(n_classes):
    target_names.append(clsnames.split(', ')[i])
    labels.append(int(i+1))

# seed -> score
# 7487 -> 91.67, cv90

#seed_rand = 7487
seed_rand = random.randint(0, 9999)

test_split = 0.2

fold = 10
repeat = 1

X = feat_in[..., 2:]
y, processing_id = feat_in[..., 1], feat_in[..., 0]

# processing_id = classnum + file_num
X_train, X_test, y_train, y_test, prid_train, prid_test = train_test_split(
    X, y, processing_id, test_size=test_split, random_state=seed_rand, stratify=y)

prid_test_files = []
test_class_list = []

In [78]:
#
# process processing_id string to extract class and file number
#
for i, e in enumerate(prid_test):
  # print(i)
  prid = int(prid_test[i])
  prid_str = str(prid)
  class_number = str(int(prid_str[0]))
  test_class_list.append(class_number)
  file_number = prid_str[1:]

  found = 0
  for i in range(n_classes):
    # print(str(i)+": "+str(labels[i])+", "+class_number+"\n")
    if (class_number == str(int(labels[i]))):
        class_name = target_names[i]
        found = 1

  if (not found):
    print ("Wrong class - {}".format(class_number))
    sys.exit(0)

  filename = class_name + " (" + file_number + ")"
  prid_test_files.append(filename.lower())

In [107]:

def evaluate_clf(clfload, clf_name, feat_file, clf, clfrep, X, y, X_train, y_train, X_test, y_test, y_pred):
    #
    # method for classifier evaluation and reporting
    #
    clf_report = metrics.classification_report(
        y_test, y_pred, labels=labels, target_names=target_names)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    conf = metrics.confusion_matrix(y_test, y_pred)

    print("\n Feature file ::: \n {}".format(feat_file))
    print("\n Feature file ::: \n {}".format(feat_file),clfrep)

    print("\n \n Classifier ::: \n {}".format(clf))
    print("\n \n Classifier ::: \n {}".format(clf),clfrep)

    print("\n \n Classification report ::: \n{}".format(clf_report))
    print("\n \n Classification report ::: \n{}".format(clf_report),clfrep)

    table = prt(["Seed", "Test split"])
    table.add_row([seed_rand, test_split])

    print("\n \n {}".format(table))
    print("\n \n {}".format(table),clfrep)

    table = prt(["Accuracy (%)"])
    table.add_row([round(accuracy, 4) * 100])

    print("\n \n {}".format(table))
    print("\n \n {}".format(table),clfrep)

    print("\n \n Confusion matrix ::: \n {} \n".format(conf))
    print("\n \n Confusion matrix ::: \n {} \n".format(conf),clfrep)

    #
    # map test files to prediction
    #
    table = prt(["Test file", "Real class", "Predicted class"])
    table.align["Test file"] = "l"
    table.align["Real class"] = "c"
    table.align["Predicted class"] = "c"

    for i, e in enumerate(X_test):
        testfile = prid_test_files[i]
        realclass = target_names[int(test_class_list[i]) - 1]
        predclass = target_names[int(y_pred[i]) - 1]

        # marking misclassified data
        if (realclass != predclass):
            table.add_row([testfile, realclass, predclass])

    print("\n Misclassified samples :::")
    print("\n Misclassified samples :::",clfrep)

    print("\n {}".format(table.get_string(sortby="Test file")))
    print("\n {}".format(table.get_string(sortby="Test file")),clfrep)

    # print SVM best parameters from GridSearchCV
    svm_best_param = 0
    if (clf_name == "SVM-RBF-"):
        if not clfload:
            svm_best_score = clf.best_score_
            svm_best_param = clf.best_params_

            print("\n \n Best: {} using {}".format(
                svm_best_score, svm_best_param))
            print("\n \n Best: {} using {}".format(
                round(svm_best_score, 5), svm_best_param))
    #
    # classwise accuracy calculation
    #
    cmat, classwise_acc, accu, clf_new = eval_classwise(
        clf_name, feat_file, clf, clfrep, X, y, svm_best_param)

    table = prt(["Class", "Accuracy (%)"])
    table.align["Class"] = "l"
    table.align["Accuracy (%)"] = "c"

    classwise_acc_avg = []

    for i in range(n_classes):
        cls_i_acc_avg = round(classwise_acc[i], 4) * 100
        classwise_acc_avg.append(cls_i_acc_avg)
        table.add_row([target_names[i], cls_i_acc_avg])

    print("\n {}".format(table))
    print("\n {}".format(table),clfrep)

    #
    # plot classwise accuracy
    #
    x_pos = np.arange(len(target_names))
    y_pos = range(0, 105, 5)
    perf = classwise_acc_avg

    my_cmap = cm.get_cmap('Blues')
    plt.clf()
    plt.bar(x_pos, perf, align='center', width=0.2, color=my_cmap(perf))

    plt.ylabel('Average accuracy')
    plt.xlabel('Class')
    plt.xticks(x_pos, target_names)
    plt.yticks(y_pos, y_pos)
    plt.title('Class-wise accuracy rate')
    plt.plot()
    plt.savefig(clfout + '/' + 'CLF-' + timestamp + '-classwise accuracy.pdf')

    #
    # cross validate with new optimized classifier
    #
    clf = clf_new
    cv=RepeatedStratifiedKFold( n_splits=fold, n_repeats=repeat,random_state=36851234)
    cv_score = cross_val_score(
        clf, X, y,cv=cv )

    table = prt(["Mean of " + str(repeat) + " times " + str(fold) +
                 "-fold CV accuracy (%)"])
    table.add_row([round(cv_score.mean(), 4) * 100])

    print("\n \n {}".format(table))
    print("\n \n {}".format(table),clfrep)

    # cleanup
    clfrep.close()
    with open(clfout + '/' + 'CLF-' + timestamp + '.pkl', 'wb') as outfile:
        cPickle.dump(clf, outfile, protocol=cPickle.HIGHEST_PROTOCOL)

    clf_time = time.time() - clf_gen  # timing finished

    print('\n Classification finished in::: %.2fs.' % clf_time)

    return [(round(accuracy, 4) * 100), (round(cv_score.mean(), 4) * 100)]
    # method finished


In [94]:

def eval_classwise(clf_name, feat_file, clf, clfrep, X, y, svm_best_param):
    cmat = []
    accu = []

    # new SVM classifier with optimized parameters
    if (clf_name == "SVM-RBF-"):
        if not clfload:
            old_clf = clf
            clf = svm.SVC(C=svm_best_param["C"], gamma='scale', kernel='rbf')

    #
    # manual m-times k-fold CV
    #
    for i in range(repeat):
        # random.seed(i**2)
        #seed_rand = random.randint(0, 999)

        skf = StratifiedKFold(fold, shuffle=True, random_state=seed_rand)

        for train_idx, test_idx in skf.split(X, y):
            X_train = X[train_idx]
            X_test = X[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]

            clf = clf.fit(X_train, y_train)

            # new XGB prediction with optimized parameters
            if (clf_name == "XGB-"):
                y_pred = clf.predict(X_test)
            else:
                y_pred = clf.predict(X_test)

            acc = metrics.accuracy_score(y_test, y_pred, normalize=True)
            accu.append(acc)

            # calculating confusion matrix every round
            conf = metrics.confusion_matrix(y_test, y_pred)
            cm = conf.astype("float").diagonal() / conf.sum(axis=1)
            cmat.append(cm)

    cmat = np.ravel(cmat)
    cmat = np.reshape(cmat, (-1, n_classes))

    classwise_acc = []

    for i in range(n_classes):
        cls_i_acc = np.array(cmat)[:, i]
        cls_i_acc_avg = sum(cls_i_acc) / len(cls_i_acc)
        classwise_acc.append(cls_i_acc_avg)

    return cmat, classwise_acc, accu, clf


In [81]:


if (svm_f):
    print("\n Beginning image classification... \n")
    clf_gen = time.time()

    clf_name = "SVM-RBF-"

    clfout = os.path.join(outfolder + 'CLF-' + clf_name + timestamp)
    os.makedirs(clfout)

    clfrep = open(clfout + '/' + 'Classification_Report-' +
                  timestamp + '.txt', 'w')

    # SVM
    clfload = 0
    if clfload:
        # load classifier
        clfpath = glob.glob(
            outfolder + "CLF-SVM-RBF-2020-01-01_00-00-06/CLF-2020-01-01_00-00-06.pkl")

        clfpath = str(clfpath[0])
        with open(clfpath, "rb") as clfin:
            clf = cPickle.load(clfin)
    else:
        # create classifier
        # gamma ='scale' => 1 / (n_features * X.var())
        svc = svm.SVC(gamma='scale', kernel='rbf', cache_size=100)
        # parameter tuning
        param_grid = [{'C': [2**9, 2**10, 2**15, 2**20]}]
        clf = GridSearchCV(svc, param_grid,
                           cv=StratifiedKFold(fold), iid=False, verbose=0, error_score=np.nan)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    SVM_ACC, SVM_CV = evaluate_clf(clfload, clf_name, feat_file,
                                   clf, clfrep, X, y, X_train, y_train, X_test, y_test, y_pred)

### Label Encoding

In [141]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y = le.transform(y)
# prid_test = le.transform(prid_test)

In [86]:

print("\n Beginning image classification... \n")
clf_gen = time.time()

clf_name = "XGB-"

clfout = os.path.join(outfolder + 'CLF-' + clf_name + timestamp)
os.makedirs(clfout)

clfrep = open(clfout + '/' + 'Classification_Report-' +
              timestamp + '.txt', 'w')



 Beginning image classification... 



In [ ]:

# xGBoost
clfload = 0
if clfload:
    # load classifier
    clfpath = glob.glob(
        outfolder + "CLF-XGB-2020-01-01_00-00-06/CLF-2020-01-01_00-00-06.pkl")

    clfpath = str(clfpath[0])
    with open(clfpath, "rb") as clfin:
        clf = cPickle.load(clfin)
else:
    clf = xgb.XGBClassifier(min_child_weight=5)

eval_set = [(X_test, y_test)]
clf = clf.fit(X_train, y_train, early_stopping_rounds=10,
              eval_metric="mlogloss", eval_set=eval_set, verbose=0)

y_pred = clf.predict(X_test)

#
# extracting feature columns (from file)
#
# processing file name string
fcols = fcols.replace('\'', '')
fcols = fcols.replace('[', '')
fcols = fcols.replace(']', '')

fc = []
for i in range(feat_in.shape[1]):
    a = fcols.split(', ')[i]
    fc.append(a)

fcols = fc[2:]  # cut out first two columns (prid, class)

impr = clf.feature_importances_
impr = np.array(impr)
# find largest n features
impr_sort = impr.argsort()[-20:][::-1]

imp_feat = []
# map important features to name
for i, e in enumerate(impr_sort):
    imp_feat.append(fcols[impr_sort[i]])

imp_f = imp_feat[::-1]  # reverse the list (descending order)


In [ ]:
# dump the XGB model
clf.get_booster().dump_model(clfout + '/' + 'CLF-' + timestamp +
                              '-xgb-dump.txt', with_stats=True, dump_format='text')

In [99]:
# plot feature importance by gain
plt.clf()
xgb.plot_importance(clf, max_num_features=20,
                    importance_type='gain', show_values=False, grid=False)

plt.yticks(range(len(imp_feat)), imp_f, rotation=45)
plt.tick_params(axis='x', which='major', labelsize=8)
plt.tick_params(axis='y', which='major', labelsize=4)
plt.xlabel("Gain")
plt.ylabel("Features")
plt.plot()
plt.savefig(clfout + '/' + 'CLF-' + timestamp + '-feature importance.pdf')

In [108]:
XGB_ACC, XGB_CV = evaluate_clf(clfload, clf_name, feat_file,
                                clf, clfrep, X, y, X_train, y_train, X_test, y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.


 Feature file ::: 
 FL-2021-05-01_16-09-52

 Feature file ::: 
 FL-2021-05-01_16-09-52 <_io.TextIOWrapper name='/content/drive/MyDrive/og_cropped/output/CLF-XGB-2023-10-26_07-28-22/Classification_Report-2023-10-26_07-28-22.txt' mode='w' encoding='UTF-8'>

 
 Classifier ::: 
 XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tre

<ipython-input-107-b9645fe9d55e>:95: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  my_cmap = cm.get_cmap('Blues')



 
 +-----------------------------------------+
| Mean of 1 times 10-fold CV accuracy (%) |
+-----------------------------------------+
|                  83.33                  |
+-----------------------------------------+

 
 +-----------------------------------------+
| Mean of 1 times 10-fold CV accuracy (%) |
+-----------------------------------------+
|                  83.33                  |
+-----------------------------------------+ <_io.TextIOWrapper name='/content/drive/MyDrive/og_cropped/output/CLF-XGB-2023-10-26_07-28-22/Classification_Report-2023-10-26_07-28-22.txt' mode='w' encoding='UTF-8'>

 Classification finished in::: 5463.21s.


In [109]:

# final report
print ("\n \n Summing up...")

table = prt(["*****", "Support Vector Machine", "Extreme Gradient Boosting"])

table.add_row(["Accuracy", SVM_ACC, XGB_ACC])
table.add_row([str(fold) + "-fold CV", SVM_CV, XGB_CV])
print("\n \n {}".format(table))


# finishing up
armageddon = time.time() - genesis  # timing finished

print('\n Total time: %.3fs.' % armageddon)
print("\n END OF CODE")



 
 Summing up...

 
 +------------+------------------------+---------------------------+
|   *****    | Support Vector Machine | Extreme Gradient Boosting |
+------------+------------------------+---------------------------+
|  Accuracy  |          0.0           |           95.83           |
| 10-fold CV |          0.0           |           83.33           |
+------------+------------------------+---------------------------+

 Total time: 5623.341s.

 END OF CODE
